In [17]:
import os
import streamlit as st
import fitz
from dotenv import load_dotenv
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display, update_display

In [2]:
load_dotenv(override = True)
openai_api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()

In [3]:
pdf_folder = r"C:\Users\User\Documents\Projects\llm_engineering\PDF-chatbot\books"

In [18]:
def extract_text():
    all_text = []
    for filename in os.listdir(r"C:\Users\User\Documents\Projects\llm_engineering\PDF-chatbot\books"):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder, filename)
            doc = fitz.open(pdf_path)
            text = ''
            for page in doc:
                content = page.get_text('text')
                text += content
            all_text.append(text)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size =500, chunk_overlap=100)
    docs = text_splitter.create_documents(all_text)
    embeddings = OpenAIEmbeddings()
    vector_store = Chroma.from_documents(docs, embeddings)   
    return vector_store

In [19]:
vector_store = extract_text()

In [20]:

def bot(query):
    llms = ChatOpenAI(model = 'gpt-3.5-turbo')
    rag_chain = RetrievalQA.from_chain_type(llm=llms, retriever =vector_store.as_retriever())
    talk = rag_chain.invoke(query)
    response = talk['result']
    return response

In [21]:
def system_prompt_func(response):
    system_prompt = 'you are an AI assitant who is very  knowlegeable and smart \n'
    system_prompt += f'You are to compare the user input and the response. this is the response:  {response}\n'
    system_prompt += "if the response does not fit the user's needs, find a better answer. If the response is good, respond with just the same answer"
    return system_prompt

In [9]:
system_prompt_func('I dont know')

"you are an AI assitant who is very  knowlegeable and smart \nYou are to compare the user input and the response. this is the response:  I dont know\nif the response does not fit the user's needs, find a better answer. If the response is good, respond with just the same answer"

In [22]:
def user_prompt_func(query,response):
    user_prompt = "compare the query and the response and confirm if the response is suitable. \n"
    user_prompt = 'If it is not suitable provide a better answer'
    user_prompt+= f'query : {query}, response: {response}'
    return user_prompt

In [25]:
st.title("Multi-PDF Chatbot for Exam Prep")
st.markdown("Ask anything from the loaded PDFs or infact, anything else...")

2025-03-24 00:36:09.260 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 00:36:09.524 
  command:

    streamlit run C:\Users\User\anaconda3\envs\llms\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-24 00:36:09.524 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 00:36:09.524 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 00:36:09.540 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [26]:
query = st.text_input('what do you need help with?')

2025-03-24 00:44:25.814 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 00:44:25.814 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 00:44:25.814 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 00:44:25.814 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 00:44:25.831 Session state does not function when running a script without `streamlit run`
2025-03-24 00:44:25.832 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-24 00:44:25.836 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [23]:
def assistant():
    response = bot(query)
    chat_bot = openai.chat.completions.create(
         model='gpt-3.5-turbo',
        messages=[
            {"role": "system", "content": system_prompt_func(response)},
            {"role": "user", "content": user_prompt_func(query,response)}
      ],
    )
    result = chat_bot.choices[0].message.content
    st.write(result)
    
    

In [24]:
assistant()

what do you need help with? who is responsible for rad 420


The response is already suitable and accurate. Professor C. C. Ohagwu from the Department of Radiography and Radiological Sciences at Nnamdi Azikiwe University is responsible for RAD 420, which involves radiographs of the spine.